# Google drive mount

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 파일 가져오기

In [2]:
cd /content

/content


In [3]:
# /content/drive/MyDrive/rokey
!unzip /content/drive/MyDrive/rokey/rokey-boot-camp-mini-competition-2.zip # ******를 자신의 경로에 맞게 수정하기


Archive:  /content/drive/MyDrive/rokey/rokey-boot-camp-mini-competition-2.zip
replace Kaggle02/class_name.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# 파일 구조 확인

In [13]:
!ls Kaggle02/


class_name.json  test  train


In [14]:
!ls Kaggle02/train

Appliance  Clothing    Fish	  GeologicalFormation  MusicalInstrument  SportsEquipment  Utensil
Bird	   Container   Food	  Insect	       Plant		  Structure	   Vehicle
Building   Electronic  Furniture  Mammal	       Reptile		  Tool		   Weapon


In [15]:
!ls Kaggle02/test

000000.JPEG  000750.JPEG  001500.JPEG  002250.JPEG  003000.JPEG  003750.JPEG  004500.JPEG
000001.JPEG  000751.JPEG  001501.JPEG  002251.JPEG  003001.JPEG  003751.JPEG  004501.JPEG
000002.JPEG  000752.JPEG  001502.JPEG  002252.JPEG  003002.JPEG  003752.JPEG  004502.JPEG
000003.JPEG  000753.JPEG  001503.JPEG  002253.JPEG  003003.JPEG  003753.JPEG  004503.JPEG
000004.JPEG  000754.JPEG  001504.JPEG  002254.JPEG  003004.JPEG  003754.JPEG  004504.JPEG
000005.JPEG  000755.JPEG  001505.JPEG  002255.JPEG  003005.JPEG  003755.JPEG  004505.JPEG
000006.JPEG  000756.JPEG  001506.JPEG  002256.JPEG  003006.JPEG  003756.JPEG  004506.JPEG
000007.JPEG  000757.JPEG  001507.JPEG  002257.JPEG  003007.JPEG  003757.JPEG  004507.JPEG
000008.JPEG  000758.JPEG  001508.JPEG  002258.JPEG  003008.JPEG  003758.JPEG  004508.JPEG
000009.JPEG  000759.JPEG  001509.JPEG  002259.JPEG  003009.JPEG  003759.JPEG  004509.JPEG
000010.JPEG  000760.JPEG  001510.JPEG  002260.JPEG  003010.JPEG  003760.JPEG  004510.JPEG
000011.JPE

In [24]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import pandas as pd
from glob import glob
import json
from collections import defaultdict
import pandas as pd
import copy
import torchvision.models as models
import random
from functools import partial
from typing import Any, Callable, List, Optional, Type, Union
import cv2
import numpy as np
from torchvision.transforms import ToPILImage
import torch
import torch.nn as nn
from torch import Tensor

from PIL import ImageFilter
import torchvision.transforms as transforms

In [32]:
class CLAHETransform:
    def __call__(self, img):
        img = np.array(img)
        img_yuv = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        clahe = cv2.createCLAHE(clipLimit=1.5, tileGridSize=(8, 8))
        img_yuv[:, :, 0] = clahe.apply(img_yuv[:, :, 0])
        img = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)
        return ToPILImage()(img)
class HistogramEqualization:
    def __call__(self, img):
        img = np.array(img)
        img_yuv = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        img_yuv[:, :, 0] = cv2.equalizeHist(img_yuv[:, :, 0])
        img = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)
        return ToPILImage()(img)
class SharpenTransform:
    def __call__(self, img):
        return img.filter(ImageFilter.SHARPEN)


# DataLoader

압축푼 직후에는 파일적용이 되지 않아 FileNotFoundError 오류가 뜰 수 있습니다.

약간 대기 후 다시 실행하면 됩니다.

In [33]:
class MyDataset(Dataset):
    def __init__(self, data_path, transform=None, train=True):
        self.train = train
        with open(f"{data_path}/class_name.json") as f:
            self.class_name = json.load(f)

        if self.train:
            self.img_path = glob(f"{data_path}/train/*/*.JPEG")
            self.labels =  [self.class_name[d.split("/")[-2]] for d in self.img_path]
        else:
            self.img_path = glob(f"{data_path}/test/*.JPEG")

        self.transform = transform

    def __len__(self):
        return len(self.img_path)

    def __getitem__(self, index):
        img = Image.open(self.img_path[index])
        if img.mode != 'RGB':
            img = img.convert('RGB')


        if self.transform:
            img = self.transform(img)

        if self.train:
            return img, self.labels[index]
        else:
            return img, self.img_path[index].split("/")[-1]

data_path = "./Kaggle02"

# 이재현님 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 이미지 크기 조정
    transforms.RandomHorizontalFlip(),  # 좌우 반전
    transforms.RandomRotation(10),  # 회전
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # 밝기, 대비, 채도 변형

    transforms.ToTensor(),
])

# # # 이선우 전처리
# transform = transforms.Compose([
#     CLAHETransform(),           # 명암 대비 증가
#     HistogramEqualization(),     # 밝기 분포 평활화
#     SharpenTransform(),          # 최종적으로 선명도 증가
#     transforms.ToTensor(),
#         transforms.RandomHorizontalFlip(p=0.2),  # 좌우 반전 (50% 확률)
#     transforms.RandomVerticalFlip(p=0.2),    # 상하 반전 (50% 확률)
#     transforms.RandomRotation(10),      # -10도 ~ +10도 범위에서
#     transforms.Resize((224, 224)),           # 크기 조정: 원상복귀
#     # transforms.RandomHorizontalFlip(p=0.5),  # 좌우 반전
#     # transforms.RandomVerticalFlip(p=0.5),    # 상하 반전
#     # transforms.RandomCrop(size=(64, 64)),  # 자르기
#     # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # 색상 조정
#     # transforms.Normalize(mean=mean, std=std),
# ])

# train_data = MyDataset(data_path, train=True, transform=transform)
test_data = MyDataset(data_path, train=False, transform=transform)

# Split train data into train and validation
train_size = int(len(train_data) * 0.9)

train_data, val_data = torch.utils.data.random_split(train_data, [train_size, len(train_data) - train_size])
val_data_copy = copy.deepcopy(val_data)

batch_size = 64
# # ---------------------------------------------------------------------
# train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

# ## 평균과 표준편차 초기화
# mean = 0.0
# std = 0.0
# total_images = 0

# for images, _ in train_loader:
#     # 현재 배치의 평균과 표준편차 계산
#     batch_mean = torch.mean(images, dim=[0, 2, 3])
#     batch_std = torch.std(images, dim=[0, 2, 3])

#     # 전체 평균과 표준편차 업데이트
#     mean += batch_mean * images.size(0)
#     std += batch_std * images.size(0)
#     total_images += images.size(0)

# # 최종 평균과 표준편차 계산
# mean /= total_images
# std /= total_images
# std *= 4

# print(f"Mean: {mean}, Std: {std}")

# # 정규화
# transform = transforms.Compose([
#     *train_data.dataset.transform.transforms,  # 기존 변환을 풀어서 포함
#     transforms.Normalize(mean=mean, std=std), # 새로 추가할 변환
# ])
# --------------------------------------------

transform_test = transforms.Compose([
    CLAHETransform(),           # 명암 대비 증가
    HistogramEqualization(),     # 밝기 분포 평활화
    SharpenTransform(),          # 최종적으로 선명도 증가
    transforms.ToTensor(),

    transforms.Resize((224, 224)),           # 크기 조정: 원상복귀
    # transforms.Normalize(mean=mean, std=std), # 새로 추가할 변환
])

train_data.dataset.transform = transform
val_data_copy.dataset.transform = transform_test
test_data.transform = transform_test

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data_copy, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

학습 잘안됨 : ****tool****, ****electronic****, ****fumiture****, ****container****, utensil, structure, sportsequipment, MusicalInstrument
    
학습 너무 잘됨 :  ****GeologicalFormation****, ****Building****, ****Food****, ****Mammal****,Insect, Bird , Reptile,


In [30]:
import warnings
import matplotlib.pyplot as plt

bad_list = [11, 13, 19, 10]
bad_words = ['tool', 'electronic', 'fumiture', 'container']
bad_dict = dict(zip(bad_list, bad_words))

good_list = [18, 6, 8, 16]
good_words = ['GeologicalFormation', 'Building', 'Food', 'Mammal']
good_dict = dict(zip(good_list, good_words))

In [36]:
# 이선우 전처리
import warnings
import matplotlib.pyplot as plt

# 모든 경고 무시
warnings.filterwarnings("ignore")

# 이미지 시각화 함수
def display_images(image_list, title, label_dict, num_images=40, width=5):
    rows = (num_images + width - 1) // width  # 필요한 행 수 계산
    fig, axes = plt.subplots(rows, width, figsize=(15, 3 * rows))  # 행 수에 따라 크기 조정
    fig.suptitle(title)

    # 필요한 개수만큼 이미지 출력
    for i, (img, label) in enumerate(image_list[:num_images]):
        ax = axes[i // width, i % width]  # 위치 계산

        # 이미지 데이터 클리핑 (0-1 범위)
        img = img.clamp(0, 1)  # 0과 1 사이로 클리핑 (0-1 범위로 정규화된 경우)
        # 또는
        # img = img.clip(0, 255)  # 0-255 범위로 클리핑 (정수형 데이터의 경우)

        ax.imshow(img.permute(1, 2, 0))  # (C, H, W) -> (H, W, C) 형식으로 변경
        ax.axis('off')
        ax.set_title(label_dict[label])  # 딕셔너리에서 라벨에 해당하는 단어 가져오기

    # 남은 서브플롯은 비워두기
    for j in range(i + 1, rows * width):
        ax = axes[j // width, j % width]
        ax.axis('off')

    plt.show()

# 조건에 맞는 이미지 필터링 함수 (랜덤 시작점 및 순환 방식)
def filter_images_by_label(dataset, label_list, num_images):
    filtered_images = []
    count_per_label = {label: 0 for label in label_list}
    remaining_labels = set(label_list)

    # 무작위 시작점 선택
    start_index = random.randint(0, len(dataset) - 1)
    dataset_size = len(dataset)

    # 데이터셋을 시작 인덱스부터 순환
    idx = start_index
    while remaining_labels:
        img, label = dataset[idx]

        # 라벨이 label_list에 있고, 아직 num_images에 도달하지 않았다면 추가
        if label in remaining_labels:
            if count_per_label[label] < num_images:
                filtered_images.append((img, label))
                count_per_label[label] += 1

            # 해당 라벨이 충분한 이미지 수를 얻었으면 제거
            if count_per_label[label] >= num_images:
                remaining_labels.remove(label)

        # 인덱스 증가, 끝에 도달하면 처음으로 돌아감
        idx = (idx + 1) % dataset_size

    return filtered_images
number = 14
width = 7
# good과 bad 리스트에 따라 이미지 필터링
good_images = filter_images_by_label(train_data, good_list, number)
bad_images = filter_images_by_label(train_data, bad_list, number)

# good 이미지 내에서 라벨 기준으로 정렬
good_images_sorted = sorted(good_images, key=lambda x: x[1])  # 라벨 기준으로 정렬

# bad 이미지 내에서 라벨 기준으로 정렬
bad_images_sorted = sorted(bad_images, key=lambda x: x[1])  # 라벨 기준으로 정렬

# good과 bad 이미지 출력
display_images(good_images_sorted, "Good Images", good_dict, num_images=4 * number, width=width)  # 넓이 5
print("-" * 100)
display_images(bad_images_sorted, "Bad Images", bad_dict, num_images=4 * number, width=width)  # 넓이 5


Output hidden; open in https://colab.research.google.com to view.

In [12]:
print(train_data.dataset.transform)
print(val_data_copy.dataset.transform)
print(test_data.transform)

print(train_loader.dataset.dataset.transform)
print(val_loader.dataset.dataset.transform)
print(test_loader.dataset.transform)


Compose(
    ToTensor()
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
)
Compose(
    ToTensor()
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
)
Compose(
    ToTensor()
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
)
Compose(
    ToTensor()
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
)
Compose(
    ToTensor()
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
)
Compose(
    ToTensor()
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
)


In [ ]:
# 이재현님 전처리
import warnings
import matplotlib.pyplot as plt

# 모든 경고 무시
warnings.filterwarnings("ignore")

# 이미지 시각화 함수
def display_images(image_list, title, label_dict, num_images=40, width=5):
    rows = (num_images + width - 1) // width  # 필요한 행 수 계산
    fig, axes = plt.subplots(rows, width, figsize=(15, 3 * rows))  # 행 수에 따라 크기 조정
    fig.suptitle(title)

    # 필요한 개수만큼 이미지 출력
    for i, (img, label) in enumerate(image_list[:num_images]):
        ax = axes[i // width, i % width]  # 위치 계산

        # 이미지 데이터 클리핑 (0-1 범위)
        img = img.clamp(0, 1)  # 0과 1 사이로 클리핑 (0-1 범위로 정규화된 경우)
        # 또는
        # img = img.clip(0, 255)  # 0-255 범위로 클리핑 (정수형 데이터의 경우)

        ax.imshow(img.permute(1, 2, 0))  # (C, H, W) -> (H, W, C) 형식으로 변경
        ax.axis('off')
        ax.set_title(label_dict[label])  # 딕셔너리에서 라벨에 해당하는 단어 가져오기

    # 남은 서브플롯은 비워두기
    for j in range(i + 1, rows * width):
        ax = axes[j // width, j % width]
        ax.axis('off')

    plt.show()

# 조건에 맞는 이미지 필터링 함수
def filter_images_by_label(dataset, label_list, num_images):
    filtered_images = []
    count_per_label = {label: 0 for label in label_list}
    remaining_labels = set(label_list)  # 아직 조건을 충족하지 않은 라벨만을 추적

    for img, label in dataset:
        # 라벨이 label_list에 있는지 확인하고 조건을 만족하지 않은 경우만 처리
        if label in remaining_labels:
            if count_per_label[label] < num_images:
                filtered_images.append((img, label))
                count_per_label[label] += 1

            # 라벨이 원하는 개수만큼 채워졌으면 제외
            if count_per_label[label] >= num_images:
                remaining_labels.remove(label)

        # 모든 라벨이 num_images만큼 채워졌다면 반복 종료
        if not remaining_labels:
            break

    return filtered_images
number = 14
width = 7
# good과 bad 리스트에 따라 이미지 필터링
good_images = filter_images_by_label(train_data, good_list, number)
bad_images = filter_images_by_label(train_data, bad_list, number)

# good 이미지 내에서 라벨 기준으로 정렬
good_images_sorted = sorted(good_images, key=lambda x: x[1])  # 라벨 기준으로 정렬

# bad 이미지 내에서 라벨 기준으로 정렬
bad_images_sorted = sorted(bad_images, key=lambda x: x[1])  # 라벨 기준으로 정렬

# good과 bad 이미지 출력
display_images(good_images_sorted, "Good Images", good_dict, num_images=4 * number, width=width)  # 넓이 5
print("-" * 100)
display_images(bad_images_sorted, "Bad Images", bad_dict, num_images=4 * number, width=width)  # 넓이 5


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
print(train_data.dataset.transform)
print(val_data_copy.dataset.transform)
print(test_data.transform)

print(train_loader.dataset.dataset.transform)
print(val_loader.dataset.dataset.transform)
print(test_loader.dataset.transform)


Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    RandomHorizontalFlip(p=0.5)
    RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
    ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=(0.8, 1.2), hue=(-0.2, 0.2))
    ToTensor()
)
Compose(
    ToTensor()
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
)
Compose(
    ToTensor()
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
)
Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    RandomHorizontalFlip(p=0.5)
    RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
    ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=(0.8, 1.2), hue=(-0.2, 0.2))
    ToTensor()
)
Compose(
    ToTensor()
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
)
Compose(
    ToTe

In [ ]:
# #새로 추가한 코드-----
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),  # 이미지 크기 조정
#     transforms.RandomHorizontalFlip(),  # 좌우 반전
#     transforms.RandomRotation(10),  # 회전
#     transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # 밝기, 대비, 채도 변형
#     transforms.ToTensor(),
# ])
# #--------------------
# train_data = MyDataset(data_path, train=True, transform=transform)
# test_data = MyDataset(data_path, train=False, transform=transform)
# #새로 추가한 코드-----
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# train_idx, val_idx = next(iter(skf.split(list(range(len(train_data))), train_data.labels)))
# #--------------------